# Lab 5

In [313]:
import matplotlib.pyplot as plt
import numpy as np

import seaborn as sns
import torch 
from torch import nn

## Task 1. Write ResNet architecture by pytorch.

I will try implement this [ResNet](https://arxiv.org/pdf/1512.03385).

### 1. Implement ResNet and bottleneck block

* ResNet block

In [314]:
class ResBlock(nn.Module):
    """one block of resnet_18 or resnet_34 model

    Args:
        in_out_channels (int): input and output channels
        expansion (int): expansion output channel
    """
    def __init__(self, in_out_channels, expansion=1):
        super().__init__()
        layers = list()  

        layers.append(nn.Conv2d(in_out_channels, in_out_channels, kernel_size=3, stride=1, padding=1))
        layers.append(nn.BatchNorm2d(in_out_channels))
        layers.append(nn.ReLU())
        
        layers.append(nn.Conv2d(in_out_channels, in_out_channels*expansion, kernel_size=3, stride=1, padding=1))
        layers.append(nn.BatchNorm2d(in_out_channels*expansion))
        
        self.layers = nn.Sequential(*layers)
        self.activastion = nn.ReLU()
        
        # if dimension is up
        if in_out_channels != in_out_channels*expansion:
            w = list()
            w.append(nn.Conv2d(in_out_channels, in_out_channels*expansion, kernel_size=3, stride=1, padding=1))
            w.append(nn.BatchNorm2d(in_out_channels*expansion))
            self.weights = nn.Sequential(*w)
        else:
            self.weights = lambda x: x
        
    def forward(self, x):
        return self.activastion( self.layers(x) + self.weights(x) )

* BottleNeck block

In [315]:
class BottleNeck(nn.Module):
    """BottleNeck block for resnet_50 model or more layers version

    Args:
        in_out_channels (int): input and output channels
        expansion (int): expansion output channel
    """
    def __init__(self, in_out_channels, expansion=4):
        super().__init__()
        layers = list()  
            
        layers.append(nn.Conv2d(in_out_channels, in_out_channels, kernel_size=1, stride=1))
        layers.append(nn.BatchNorm2d(in_out_channels))
        layers.append(nn.ReLU())
        
        layers.append(nn.Conv2d(in_out_channels, in_out_channels, kernel_size=3, stride=1, padding=1))
        layers.append(nn.BatchNorm2d(in_out_channels))
        layers.append(nn.ReLU())
        
        layers.append(nn.Conv2d(in_out_channels, int(in_out_channels*expansion), kernel_size=1, stride=1))
        layers.append(nn.BatchNorm2d(int(in_out_channels*expansion)))
        
        self.layers = nn.Sequential(*layers)
        self.activastion = nn.ReLU()
        
        if in_out_channels != in_out_channels*expansion:
            w = list()
            w.append(nn.Conv2d(in_out_channels, int(in_out_channels*expansion), kernel_size=3, stride=1, padding=1))
            w.append(nn.BatchNorm2d(int(in_out_channels*expansion)))
            self.weights = nn.Sequential(*w)
        else:
            self.weights = lambda x: x
        
    def forward(self, x):
        return self.activastion( self.layers(x) + self.weights(x) )

### 2. Implement resnet layers

In [316]:
class convi_x(nn.Module):
    def __init__(self, in_out_channels, blocks, is_last=False, is_resblock=True):
        """_summary_
        conv2_x, conv3_x, conv4_x, conv5_x layers in one class
        Args:
            in_out_channels (int):  input and output channels
            blocks (int): num of blocks
            is_last (bool, optional): var for checking is it conv5_x. Defaults to False.
            is_resnet (bool, optional): set ups what type of connection will use: Residual or Bottleneck. Defaults to True.        
        """
        super().__init__()
        layers = list()  
               
        if is_last:
            if is_resblock:
                for i in range(blocks):           
                    layers.append(ResBlock(in_out_channels) )
            else:
                layers.append(BottleNeck(in_out_channels))
                for i in range(blocks):           
                    layers.append(BottleNeck(in_out_channels * 4, expansion = 1))
                   
        else:
            if is_resblock:
                for i in range(blocks-1):           
                    layers.append(ResBlock(in_out_channels))                        # channels n->n example input: 64; output: 64
                layers.append(ResBlock( in_out_channels, expansion = 2 ))           # channels n->2*n example input: 64; output: 64
            else:
                layers.append(BottleNeck(in_out_channels))                          # channels n->4*n example input: 64; output: 256
                for i in range(blocks-2):           
                    layers.append(BottleNeck( in_out_channels * 4, expansion = 1))  # channels 4*n->4*n example input: 256; output: 256
                layers.append(BottleNeck( in_out_channels * 4, expansion = 0.5 ))   # channels 4*n->2*n  example input: 256; output: 128
        
            
        self.layers = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.layers(x)

### 3. Implement resnet model

In [317]:
class ResNet(nn.Module):
    def __init__(self, output_channels, architecture = 18, num_blocks = None):
        """ResNet class

        Args:
            output_channels (int): output channels
            architecture (int, optional): articture that will use. Defaults to 18. 
            
            if architecure not defaults like 18, 34, 50 and etc and < 50 will use resnet block, else bottleneck
            num_blocks (list of int, optional): num of blocks for layers. Defaults to None.
        """
        super().__init__()
        
        is_resblock = architecture < 50 # to use resblock or bottleneck block
        
        if num_blocks == None:
            num_blocks = self.get_default_num_blocks(architecture)
                
        blocks = list()
        for i in range(3):
            blocks.append( (64*2**i, num_blocks[i-1], 0) ) # in_out_channels, num of blocks and bool var for conv5_x
        blocks.append((512, num_blocks[-1], 1))

        # Convolutions
        layers = list()
        
        layers.append(nn.Conv2d(3, 64, kernel_size=7, stride=2)) # conv1
        
        layers.append(nn.MaxPool2d(3, 2))
        
        for block in blocks:
            layers.append(convi_x(block[0], block[1], block[2], is_resblock))
            
        
        layers.append(nn.AdaptiveAvgPool2d(1))    
        
        self.layers = nn.Sequential(*layers)
        
        # FC layers + Softmax activation
        self.fcl = nn.Linear(512 if is_resblock else 2048, output_channels)
        self.activastion = nn.Softmax()
    
    def get_default_num_blocks(self, architecture) -> list:
        """get default list of numbers of block for defualt architecture

        Args:
            architecture (int): ResNet architecture

        Returns:
            list (int): numbers of block
        """
        num_blocks = list()
        
        if architecture == 152:
            num_blocks = [3, 8, 36, 3]
        elif architecture == 34:
            num_blocks = [3, 4, 6, 3]
        elif architecture == 50:
            num_blocks = [3, 4, 6, 3]
        elif architecture == 101:
            num_blocks = [3, 4, 23, 3]
        else:
            num_blocks = [2, 2, 2, 2]
        
        return num_blocks
    
    def forward(self, x):
        conv_out = self.layers(x)
        conv_out = torch.flatten(conv_out, start_dim=1)
        return self.activastion(self.fcl(conv_out))

Let's check is it work!

In [318]:
ResNet_18 = ResNet(1)
ResNet_50 = ResNet(1, architecture=50)
x = torch.randn(1, 3, 64, 64)

ResNet_18.forward(x)
ResNet_50.forward(x)

c:\Users\Омар\AppData\Local\Programs\Python\Python311\Lib\site-packages\torch\nn\modules\module.py:1532: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)


tensor([[1.]], grad_fn=<SoftmaxBackward0>)